In [ ]:
import glob
import os
import msgpack
from tqdm.auto import tqdm

from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import tensorflow as tf
import tensorflow.keras as keras
import keras.models as models
import keras.layers as layers
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd

In [ ]:
train_dataset_path = input('Enter the path of the train dataset: ')
label_path = input('Enter the path of the label file: ')
csv = pd.read_csv(label_path, header=None)

features = []
labels = []

for l in tqdm(list(csv.iterrows())):
    filename = os.path.join(train_dataset_path, l[1][0] + '.bin')
    features.append(np.fromfile(filename, dtype=np.uint8))
    labels.append(l[1][1])

padded  = tf.keras.preprocessing.sequence.pad_sequences(features, padding='post', maxlen=4096)

In [ ]:
expanded_data = np.expand_dims(padded, axis=2)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(128, 3, activation='relu', input_shape=expanded_data.shape[1:]))
model.add(keras.layers.MaxPooling1D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()
print(expanded_data.shape)
print(np.array(labels).shape)

In [ ]:
model.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(expanded_data, np.array(labels) , epochs=30)

In [ ]:
# 모델 평가
test_dataset_path = input('Enter the path of the test dataset: ')
test_label_path = input('Enter the path of the label file: ')
csv = pd.read_csv(test_label_path, header=None)

test_features = []
test_labels = []

for l in tqdm(list(csv.iterrows())):
    filename = os.path.join(test_dataset_path, l[1][0] + '.bin')
    test_features.append(np.fromfile(filename, dtype=np.uint8))
    test_labels.append(l[1][1])

test_padded  = tf.keras.preprocessing.sequence.pad_sequences(test_features, padding='post', maxlen=4096)

In [ ]:
loss, accuracy = model.evaluate(x=test_padded, y=np.array(test_labels))

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}%')

In [ ]:
# Save Model and Load Model for Test
model.save('models/pe_header.h5')